In [2]:
import requests
import pandas as pd
import json
from datetime import datetime
import logging

# Create a session object
session = requests.Session()

# Disable proxy settings for the session
session.proxies = {
    "http": None,
    "https": None,
    "http://": None,
    "https://": None,
}

# Configure the logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    filename="app.log",
    filemode="w",
)

force_none_proxies = {
    "http": None,
    "https": None,
}

import hashlib


def get_md5_encoded_string(phone_number, date, word):
    # Concatenate the input values
    input_string = f"{phone_number}{date}{word}"

    # Encode the string to bytes
    input_bytes = input_string.encode("utf-8")

    # Generate the MD5 hash
    md5_hash = hashlib.md5(input_bytes)

    # Convert the hash to its hexadecimal representation
    md5_hex = md5_hash.hexdigest()

    return md5_hex


# Example usage
phone_number = "18618107293"
date = datetime.now().strftime("%Y%m%d")
word = "login"
md5_encoded_string = get_md5_encoded_string(phone_number, date, word)
logging.info(md5_encoded_string)


def get_token_for_env(env="https://qah5.summerfarm.net"):
    url = f"{env}/openid?phone={phone_number}&sign={md5_encoded_string}"
    logging.info(url)
    token = session.get(url=url, timeout=12000, proxies=force_none_proxies)
    logging.info(f"token.status_code:{token.status_code}, text:{token.text}")
    try:
        token = json.loads(token.text)
        logging.info(f"token:{token}")
        return {"env": f"{env}", "token": token["data"]["token"]}
    except Exception as e:
        logging.error(f"获取Token失败:{url}, {token}, {e}")
        raise e


# "https://qah5.summerfarm.net",
# "https://devh5.summerfarm.net",
# "http://localh5.summerfarm.net",
# "https://dev2h5.summerfarm.net",

env_old = "qah5.summerfarm.net"
env_new = "dev2h5.summerfarm.net"

env_list = [
    f"https://{env_old}",
    f"https://{env_new}",
    # "http://localh5.summerfarm.net",
]

token_list = [get_token_for_env(env) for env in env_list]

logging.info(f"获取到的token:{token_list}")

In [ ]:
from sls_client import get_sls_data_by_query
from datetime import datetime, timedelta

to_time = datetime.now() - timedelta(days=0)
from_time = datetime.now() - timedelta(minutes=60 * 24)

query = """inboundFlag: POST\:/price/query/take-actual-price and "requestBody = "| 
select json_extract(split(msg,'requestBody = ')[2], '$[0]') as orderNow 
from log group by 1 order by count(1) desc limit 200"""
sku_object_list_from_sls = get_sls_data_by_query(
    from_time=from_time,
    to_time=to_time,
    query=query,
    project="k8s-log-custom-azure-aks",
    logstore="qa-mall-stdout",
)

sku_object_list_from_sls.head(2)

In [ ]:
sku_object_list = []
sku_list = []
for index, row in sku_object_list_from_sls.iterrows():
    order_now = json.loads(row["orderNow"]).get("orderNow")
    if order_now is None:
        logging.warning(f"None:{row.to_dict()}")
        continue
    sku_list.extend(order_now)

for sku in sku_list:
    sku_object_list.append({"orderNow": sku})

sku_object_list_df=pd.DataFrame(sku_object_list)
sku_object_list_df.head(2)

In [ ]:
skus_to_request = []
for i in range(0, 6):
    sku = sku_object_list[i]
    skus_to_request.append(
        {"sku": sku.get("orderNow", {}).get("sku"), "quantity": 2, "suitid": 0}
    )

print(skus_to_request)

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
import concurrent
import json
data={'orderNow': [{'sku': '596080523106', 'suitId': 0, 'quantity': 1}]}

def process_price_api():
    url = "https://dev2h5.summerfarm.net/price/query/take-actual-price"
    resposne = requests.post(url, json=data, headers={"token":"mall__f141ede2-bb24-48bb-b3e9-8a10586a4b99"})
    print(f"{datetime.now()}:{url}",resposne.status_code, resposne.text)

with ThreadPoolExecutor(max_workers=10) as executor:
    # Submit tasks to the executor
    futures = [executor.submit(process_price_api) for index in range(1,50)]
    concurrent.futures.wait(futures)

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import concurrent
import json


def get_take_price_result(env_object={}, order_now_obj={}) -> [int, str]:
    env_name = env_object["env"]
    url = f"{env_name}/price/query/take-actual-price"
    logging.info(f"making requests to env:{env_object}\nSKU:{order_now_obj}")
    started_at = int(datetime.now().timestamp() * 1000)
    data = requests.post(
        url=url,
        json=order_now_obj,
        headers={"token": env_object["token"]},
        proxies=force_none_proxies,
    ).json()
    if data is not None and "data" in data:
        data = data.get("data")
    else:
        logging.error(
            f"获取到了错误的结果:{data}, env:{env_object}, sku:{order_now_obj}"
        )
        data = []
    # logging.info(f"env:{env_object}\ndata:{data}")
    finished_at = int(datetime.now().timestamp() * 1000)
    rt = finished_at - started_at
    # logging.info(f"RT:{rt}ms")
    return rt, data


logging.info(f"长度:{len(sku_object_list_df)}")

all_sku_toke_price_map = {}

def get_single_shot_data(sku_obj):
    global all_sku_toke_price_map
    sku = sku_obj["orderNow"]["sku"]
    rt, data = get_take_price_result(
        env_object=token_list[0], order_now_obj={"orderNow": [sku_obj["orderNow"]]}
    )
    if len(data) > 0:
        all_sku_toke_price_map[sku] = data[0]
    else:
        all_sku_toke_price_map[sku] = {}

with ThreadPoolExecutor(max_workers=4) as executor:
    # Submit tasks to the executor
    futures = [executor.submit(get_single_shot_data, sku_obj) for sku_obj in sku_object_list]
    concurrent.futures.wait(futures)


print(len(all_sku_toke_price_map))
print(all_sku_toke_price_map.get("849713171106"))

In [ ]:
print(all_sku_toke_price_map.get("5466723784"))

## 开始做验证

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import concurrent
import pandas as pd
import re
import random


def get_random_6elements(num=6):
    # Get a random sample of 6 elements
    random_sample = random.sample(sku_object_list, num)
    print(random_sample)
    all_sku_list = []

    for sub_list in random_sample:
        all_sku_list.append(sub_list["orderNow"])

    return {"orderNow": all_sku_list}


def remove_http_https(input_string):
    return re.sub(r"http://|https://", "", f"{input_string}")


df = []


def process_sku(sku_info):
    global df
    sku_info = get_random_6elements(6)
    print(sku_info)
    data_of_sku = []
    df_object = {}
    for env in token_list:
        domain_name = remove_http_https(env["env"])
        rt, data_a = get_take_price_result(env, sku_info)
        data_of_sku.append(data_a)
        df_object[f"rt_of_{domain_name}"] = rt
        df_object[f"data_of_{domain_name}"] = data_a
    df_object["requested_sku_object"] = sku_info
    df.append(df_object)

with ThreadPoolExecutor(max_workers=4) as executor:
    # Submit tasks to the executor
    futures = [executor.submit(process_sku, {}) for idex in range(0,int(len(sku_object_list)))]
    concurrent.futures.wait(futures)

df = pd.DataFrame(df)
df.head(1)

In [ ]:
def is_env_new_all_correct(row: pd.Series):
    try:
        data_of_new_env_map = {}
        data_of_old_env_map = {}
        data_of_new_env = row[f"data_of_{env_new}"]
        data_of_old_env = row[f"data_of_{env_old}"]
        if len(data_of_new_env) > 0:
            for sku_data in data_of_new_env:
                data_of_new_env_map[sku_data["sku"]] = {
                    "takeActualPrice": sku_data["takeActualPrice"],
                    "price": sku_data["price"],
                }
        if len(data_of_old_env) > 0:
            for sku_data in data_of_old_env:
                data_of_old_env_map[sku_data["sku"]] = {
                    "takeActualPrice": sku_data["takeActualPrice"],
                    "price": sku_data["price"],
                }
        requested_sku_object = row["requested_sku_object"]["orderNow"]
        result = True
        for sku in requested_sku_object:
            sku = sku["sku"]
            result_new_env = data_of_new_env_map.get(sku)
            result_old_env = data_of_old_env_map.get(sku)
            result = json.dumps(result_old_env) == json.dumps(result_new_env)
            if not result:
                return f"{result},sku:{sku}, new_env:{result_new_env}, old_env:{result_old_env}"
        return f"{result}"
    except Exception as e:
        logging.error(f"error:{e}, row:{row.to_dict()}")
        return f"error:{e}"


df["is_env_new_all_correct"] = df.apply(is_env_new_all_correct, axis=1)

In [ ]:
pd.set_option("display.max_colwidth", None)

# Use the display function to show the DataFrame
from IPython.display import display

display(
    df[df["is_env_new_all_correct"] != "True"][
        [
            "is_env_new_all_correct",
            "requested_sku_object",
            f"data_of_{env_old}",
            f"data_of_{env_new}",
        ]
    ].head(10)
)

In [ ]:
df.describe()

In [ ]:
display(
    df[
        [
            "is_env_new_all_correct",
            "requested_sku_object",
            f"data_of_{env_old}",
            f"data_of_{env_new}",
        ]
    ].head(10)
)

## 判断是否全部都和单次请求待结果一样

In [ ]:
def is_all_env_align_with_single_shot(row: pd.Series):
    try:
        is_all_correct = True
        data_of_new_env = row[f"data_of_{env_new}"]
        if len(data_of_new_env) > 0:
            for sku_data in data_of_new_env:
                sku = sku_data["sku"]
                is_all_correct = (
                    all_sku_toke_price_map.get(sku)["takeActualPrice"]
                    == sku_data["takeActualPrice"]
                )
                if not is_all_correct:
                    return f"{is_all_correct}, single:{all_sku_toke_price_map.get(sku)}, env_new:{sku_data}"
        return f"{is_all_correct}"
    except Exception as e:
        logging.error(f"error:{e}, row:{row.to_dict()}")
        return f"error:{e}"


def is_old_env_align_with_single_shot(row: pd.Series):
    try:
        is_all_correct = True
        data_of_old_env = row[f"data_of_{env_old}"]
        if len(data_of_old_env) > 0:
            for sku_data in data_of_old_env:
                sku = sku_data["sku"]
                is_all_correct = (
                    all_sku_toke_price_map.get(sku)["takeActualPrice"]
                    == sku_data["takeActualPrice"]
                )
                if not is_all_correct:
                    return f"{is_all_correct}, single:{all_sku_toke_price_map.get(sku)}, env_old:{sku_data}"
        return f"{is_all_correct}"
    except Exception as e:
        logging.error(f"error:{e}, row:{row.to_dict()}")
        return f"error:{e}"


df["is_all_the_same"] = df.apply(is_all_env_align_with_single_shot, axis=1)
df["is_old_env_all_the_same"] = df.apply(is_old_env_align_with_single_shot, axis=1)
df[df["is_all_the_same"] != "True"]